### ATAC enrichment

In [2]:
library(data.table)
# library(qvalue)
library(bedr)
options(scipen = 999)



######################
#### bedr v1.0.7 ####
######################

checking binary availability...
  * Checking path for bedtools... PASS
    /home/ssm-user/anaconda3/envs/single_cell/bin/bedtools
  * Checking path for bedops... FAIL
  * Checking path for tabix... FAIL
tests and examples will be skipped on R CMD check if binaries are missing




In [4]:

for (ct in c('B', 'T8', 'T4', 'NK', 'ncM', 'cM')){

    result_path = paste("/data_volume/memento/lupus/full_analysis/memento/hits/eur_", ct,".csv", sep="")
#     result_path = paste("/data_volume/memento/lupus/full_analysis/mateqtl/outputs/asian_", ct,".csv", sep="")

    #res_raw=data.frame(fread(args[1]))
    res_raw=data.frame(fread(result_path))
    #emp=fread(args[2])
    #delta=fread(args[3])
    out= paste("/data_volume/memento/lupus/atac_enrichment/output/eur_", ct, "_atac_mem.txt", sep="")

    
    expr_genes_path =  paste("/data_volume/memento/lupus/atac_enrichment/expr_genes/eur_", ct, ".csv", sep="")
    expr_genes=read.table(expr_genes_path,header=F)

    print('gene')
    print(head(expr_genes))
    print(dim(expr_genes))


    #print(head(emp))
    #emp$FDR=qvalue(emp$Empirical)$qvalues
    #thresh=max(emp$Pvalue[which(emp$FDR < 0.1)])

    #run if not corrected by celltype
    #fdr=qvalue(res_raw$pvalue)
    #print(fdr)
    #res_raw$FDR=fdr$qvalues

    #filter FDR
    #res=res_raw[res_raw$FDR <= 0.1,]
    #snps gene statistic pvalue FDR beta qvalue



    colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta")
    #colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta","qvalue")
    #res=res_raw
    #filter for expressed genes
#     print(head(res_raw))
#     print(dim(res_raw))
#     res_expr=res_raw[res_raw$gene %in% expr_genes$V1,]
#     print(dim((res_expr)))
#     res_noexpr=res_raw[!(res_raw$gene %in% expr_genes$V1),]

#     print(dim(res_expr))
#     print(head(res_expr))
#     print(head(res_noexpr))

#     num_no=length(res_noexpr$gene)
#     nullp=runif(num_no, min = 0, max = 1)
#     res_noexpr$p.value=nullp
#     res=rbind(res_expr,res_noexpr)
#     print(head(res_expr))
#     print(dim(res_expr))
#     print(dim(res))
#     print(tail(res))
    #atac=fread('/ye/yelabstore2/10x.lupus/process.scRNAseq/eqtls/S9_lineage_groups.txt.gz')
    atac=fread('/data_volume/memento/lupus/atac_enrichment/simple_sorted_atac_lineage_groups.txt.gz')
    #names(atac)=c('chr', 'start', 'end', 'group')
    #atac.bed=data.frame(chr=atac$chr, start=atac$start, end=atac$stop, group=atac$group)
    atac.groups=names(table(atac$group))
    #atac.sorted=bedr.sort.region(atac$chr,atac$start,atac$stop)
    res =res_raw

    res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
    res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))

    #res$chr=paste('chr', sapply(strsplit(res$snp, ':'), '[', 1), sep='')
    #res$pos=as.numeric(sapply(strsplit(res$snp, ':'), '[', 2))
    res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
    res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

    #res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
    #res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))
    #res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
    #res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

    all.df=NULL
    for(g in atac.groups){
        print(g)
        peaks=atac[which(atac$group == g), ]
        peaks=paste(peaks$chr, ':', peaks$start, '-', peaks$stop, sep='')
        overlap.sig.snps=bedr(input=list(a=res.bed.sorted, b=peaks), method='intersect')

        print("here~~~~~~~~~")

        in_peak=as.numeric(res.bed.sorted %in% overlap.sig.snps)

        pvals=res$"p.value"[match(res.bed.sorted, res$id)]
        #pvals=res$"pvalue"[match(res.bed.sorted, res$id)]
        #pvals=res$"p-value"[match(res.bed.sorted, res$id)]
        mw_res=wilcox.test(pvals ~ in_peak, alternative='greater')
        df=data.frame(W=mw_res$statistic, pval=mw_res$p.value, group=g)
        all.df=rbind(all.df, df)
        print(all.df)
    }

    write.table(all.df, file=out, row.names=F, col.names=T, quote=F, sep='\t')

}

In [14]:
sum(in_peak)

[1] 0

### Gracie attempt 2

In [2]:
library(data.table)
library(qvalue)
library(bedr)
options(scipen = 999)
# args=commandArgs(TRUE)

res_raw=data.frame(fread('/data_volume/memento/lupus/gracie_eqtls/nk_metasoft_out.txt'))
#res_raw=data.frame(fread(args[1]))[-1]
#emp=fread(args[2])
#delta=fread(args[3])
out='gracie_nk.out'
#expr_genes=read.table(args[3],header=F)

print('gene')
#print(head(expr_genes))
#print(dim(expr_genes))
#print(head(emp))
#emp$FDR=qvalue(emp$Empirical)$qvalues
#thresh=max(emp$Pvalue[which(emp$FDR < 0.1)])

#run if not corrected by celltype
#fdr=qvalue(res_raw$pvalue)
#print(fdr)
#res_raw$FDR=fdr$qvalues

#filter FDR
#res=res_raw[res_raw$FDR <= 0.1,]
#snps gene statistic pvalue FDR beta qvalue


#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta")
#take in metasoft data, create atac enrichments from fixed effects
colnames(res_raw)=c("RSID","STUDY","p.value","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE","PVALUES_OF_STUDIES(Tab_delimitered)","MVALUES_OF_STUDIES(Tab_delimitered)")
res_raw$SNP=sapply(strsplit(res_raw$RSID, '_'), '[', 1)
res_raw$gene=sapply(strsplit(res_raw$RSID, '_'), '[', 2)

#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta","qvalue")
#res=res_raw
#filter for expressed genes
print(head(res_raw))
print(dim(res_raw))
#res_expr=res_raw[res_raw$gene %in% expr_genes$V1,]
#print(dim((res_expr)))
#res_noexpr=res_raw[!(res_raw$gene %in% expr_genes$V1),]

print('expr')
#print(dim(res_expr))
#print(head(res_expr))
print('no gene')
#print(head(res_noexpr))

#num_no=length(res_noexpr$gene)
#print(num_no)
#nullp=runif(num_no, min = 0, max = 1)
#nullp=rep(1,num_no)
#res_noexpr$p.value=nullp
#res=rbind(res_expr,res_noexpr)
#print(head(res_expr))
#print(dim(res_expr))
res=res_raw
print(dim(res))
print(tail(res))
#atac=fread('/ye/yelabstore2/10x.lupus/process.scRNAseq/eqtls/S9_lineage_groups.txt.gz')
#atac=fread('simple_sorted_atac_lineage_groups2.txt.gz')
atac=fread('/data_volume/memento/lupus/atac_enrichment/sorted_simple_atac_lineage_groups3.bed.gz')

#names(atac)=c('chr', 'start', 'end', 'group')
#atac.bed=data.frame(chr=atac$chr, start=atac$start, end=atac$stop, group=atac$group)
atac.groups=names(table(atac$group))
#atac.sorted=bedr.sort.region(atac$chr,atac$start,atac$stop)

res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))

#res$chr=paste('chr', sapply(strsplit(res$snp, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$snp, ':'), '[', 2))
res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

#res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))
#res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
#res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

all.df=NULL
for(g in atac.groups){
    print(g)
    peaks=atac[which(atac$group == g), ]
    peaks=paste(peaks$chr, ':', peaks$start, '-', peaks$stop, sep='')
    overlap.sig.snps=bedr(input=list(a=res.bed.sorted, b=peaks), method='intersect')
    in_peak=as.numeric(res.bed.sorted %in% overlap.sig.snps)
    pvals=res$"p.value"[match(res.bed.sorted, res$id)]
    #pvals=res$"pvalue"[match(res.bed.sorted, res$id)]
    #pvals=res$"p-value"[match(res.bed.sorted, res$id)]
    mw_res=wilcox.test(pvals ~ in_peak, alternative='greater')
    df=data.frame(W=mw_res$statistic, pval=mw_res$p.value, group=g)
    all.df=rbind(all.df, df)   
}

write.table(all.df, file=out, row.names=F, col.names=T, quote=F, sep='\t')




Warning message in fread("/data_volume/memento/lupus/gracie_eqtls/nk_metasoft_out.txt"):
“Detected 18 column names but the data has 23 columns (i.e. invalid file). Added 5 extra default column names at the end.”


[1] "gene"
                  RSID STUDY  p.value     BETA_FE    STD_FE PVALUE_RE
1 10:100043981_PYROXD2     2 0.499939  0.05168100 0.0766115  0.569163
2 10:100044928_PYROXD2     3 0.481351 -0.03292740 0.0467632  0.533360
3 10:100045000_PYROXD2     3 0.333619 -0.03689180 0.0381567  0.631411
4 10:100045147_PYROXD2     3 0.498166 -0.02550860 0.0376579  0.748616
5 10:100045166_PYROXD2     2 0.873178  0.01125270 0.0704954  0.955431
6 10:100045813_PYROXD2     2 0.909779 -0.00787023 0.0694531  0.965707
      BETA_RE    STD_RE PVALUE_RE2 STAT1_RE2 STAT2_RE2 PVALUE_BE I_SQUARE
1  0.04935050 0.0866885  0.5359850 0.4550660  0.000000        NA  21.4242
2 -0.03152080 0.0506046  0.5344880 0.4958000  0.000000        NA  14.0455
3 -0.02936590 0.0612119  0.2534790 0.9347980  0.597342        NA  60.6759
4 -0.01923900 0.0600347  0.3559570 0.4588410  0.573694        NA  60.2236
5 -0.01125750  0.201428  0.0812551 0.0254796  3.236490        NA  87.5518
6  0.00850442  0.197812  0.0812569 0.0128408  3.249090 

### Copy gracie's attempt with memento  eqtls

In [ ]:
for (ct in c('T4', 'cM', 'ncM', 'T8', 'B', 'NK')){
print(ct)
library(data.table)
library(qvalue)
library(bedr)
options(scipen = 999)
# args=commandArgs(TRUE)

res_raw=data.frame(fread(paste('/data_volume/memento/lupus/full_analysis/memento/gracie/eur_',ct, '.csv', sep='')))
#res_raw=data.frame(fread(args[1]))[-1]
#emp=fread(args[2])
#delta=fread(args[3])
out=paste('/data_volume/memento/lupus/atac_enrichment/gracie/', 'eur_matqetl_',ct, '.out', sep='')
#expr_genes=read.table(args[3],header=F)

print('gene')
#print(head(expr_genes))
#print(dim(expr_genes))
#print(head(emp))
#emp$FDR=qvalue(emp$Empirical)$qvalues
#thresh=max(emp$Pvalue[which(emp$FDR < 0.1)])

#run if not corrected by celltype
#fdr=qvalue(res_raw$pvalue)
#print(fdr)
#res_raw$FDR=fdr$qvalues

#filter FDR
#res=res_raw[res_raw$FDR <= 0.1,]
#snps gene statistic pvalue FDR beta qvalue


#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta")
#take in metasoft data, create atac enrichments from fixed effects
colnames(res_raw)=c('SNP', 'gene', 'statistic', 'p.value', 'FDR', 'beta')
# res_raw$gene=sapply(strsplit(res_raw$RSID, '_'), '[', 2)

#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta","qvalue")
#res=res_raw
#filter for expressed genes
print(head(res_raw))
print(dim(res_raw))
#res_expr=res_raw[res_raw$gene %in% expr_genes$V1,]
#print(dim((res_expr)))
#res_noexpr=res_raw[!(res_raw$gene %in% expr_genes$V1),]

print('expr')
#print(dim(res_expr))
#print(head(res_expr))
print('no gene')
#print(head(res_noexpr))

#num_no=length(res_noexpr$gene)
#print(num_no)
#nullp=runif(num_no, min = 0, max = 1)
#nullp=rep(1,num_no)
#res_noexpr$p.value=nullp
#res=rbind(res_expr,res_noexpr)
#print(head(res_expr))
#print(dim(res_expr))
res=res_raw
print(dim(res))
print(tail(res))
#atac=fread('/ye/yelabstore2/10x.lupus/process.scRNAseq/eqtls/S9_lineage_groups.txt.gz')
#atac=fread('simple_sorted_atac_lineage_groups2.txt.gz')
atac=fread('/data_volume/memento/lupus/atac_enrichment/sorted_simple_atac_lineage_groups3.bed.gz')

#names(atac)=c('chr', 'start', 'end', 'group')
#atac.bed=data.frame(chr=atac$chr, start=atac$start, end=atac$stop, group=atac$group)
atac.groups=names(table(atac$group))
#atac.sorted=bedr.sort.region(atac$chr,atac$start,atac$stop)

res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))

#res$chr=paste('chr', sapply(strsplit(res$snp, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$snp, ':'), '[', 2))
res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

#res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))
#res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
#res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

all.df=NULL
for(g in atac.groups){
    print(g)
    peaks=atac[which(atac$group == g), ]
    peaks=paste(peaks$chr, ':', peaks$start, '-', peaks$stop, sep='')
    overlap.sig.snps=bedr(input=list(a=res.bed.sorted, b=peaks), method='intersect')
    in_peak=as.numeric(res.bed.sorted %in% overlap.sig.snps)
    pvals=res$"p.value"[match(res.bed.sorted, res$id)]
    #pvals=res$"pvalue"[match(res.bed.sorted, res$id)]
    #pvals=res$"p-value"[match(res.bed.sorted, res$id)]
    mw_res=wilcox.test(pvals ~ in_peak, alternative='greater')
    df=data.frame(W=mw_res$statistic, pval=mw_res$p.value, group=g)
    all.df=rbind(all.df, df)   
}

write.table(all.df, file=out, row.names=F, col.names=T, quote=F, sep='\t')

}


### Copy gracie's attempt with mateqtl  eqtls

In [2]:
for (ct in c('T4', 'cM', 'ncM', 'T8', 'B', 'NK')){
print(ct)
library(data.table)
library(qvalue)
library(bedr)
options(scipen = 999)
# args=commandArgs(TRUE)

res_raw=data.frame(fread(paste('/data_volume/memento/lupus/full_analysis/mateqtl/outputs/eur_',ct, '_all_hg19.csv', sep='')))
#res_raw=data.frame(fread(args[1]))[-1]
#emp=fread(args[2])
#delta=fread(args[3])
out=paste('/data_volume/memento/lupus/atac_enrichment/gracie/', 'eur_matqetl_',ct, '.out', sep='')
#expr_genes=read.table(args[3],header=F)

print('gene')
#print(head(expr_genes))
#print(dim(expr_genes))
#print(head(emp))
#emp$FDR=qvalue(emp$Empirical)$qvalues
#thresh=max(emp$Pvalue[which(emp$FDR < 0.1)])

#run if not corrected by celltype
#fdr=qvalue(res_raw$pvalue)
#print(fdr)
#res_raw$FDR=fdr$qvalues

#filter FDR
#res=res_raw[res_raw$FDR <= 0.1,]
#snps gene statistic pvalue FDR beta qvalue


#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta")
#take in metasoft data, create atac enrichments from fixed effects
colnames(res_raw)=c('SNP', 'gene', 'beta','statistic', 'p.value', 'FDR')
# res_raw$gene=sapply(strsplit(res_raw$RSID, '_'), '[', 2)

#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta","qvalue")
#res=res_raw
#filter for expressed genes
print(head(res_raw))
print(dim(res_raw))
#res_expr=res_raw[res_raw$gene %in% expr_genes$V1,]
#print(dim((res_expr)))
#res_noexpr=res_raw[!(res_raw$gene %in% expr_genes$V1),]

print('expr')
#print(dim(res_expr))
#print(head(res_expr))
print('no gene')
#print(head(res_noexpr))

#num_no=length(res_noexpr$gene)
#print(num_no)
#nullp=runif(num_no, min = 0, max = 1)
#nullp=rep(1,num_no)
#res_noexpr$p.value=nullp
#res=rbind(res_expr,res_noexpr)
#print(head(res_expr))
#print(dim(res_expr))
res=res_raw
print(dim(res))
print(tail(res))
#atac=fread('/ye/yelabstore2/10x.lupus/process.scRNAseq/eqtls/S9_lineage_groups.txt.gz')
#atac=fread('simple_sorted_atac_lineage_groups2.txt.gz')
atac=fread('/data_volume/memento/lupus/atac_enrichment/sorted_simple_atac_lineage_groups3.bed.gz')

#names(atac)=c('chr', 'start', 'end', 'group')
#atac.bed=data.frame(chr=atac$chr, start=atac$start, end=atac$stop, group=atac$group)
atac.groups=names(table(atac$group))
#atac.sorted=bedr.sort.region(atac$chr,atac$start,atac$stop)

res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))

#res$chr=paste('chr', sapply(strsplit(res$snp, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$snp, ':'), '[', 2))
res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

#res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))
#res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
#res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

all.df=NULL
for(g in atac.groups){
    print(g)
    peaks=atac[which(atac$group == g), ]
    peaks=paste(peaks$chr, ':', peaks$start, '-', peaks$stop, sep='')
    overlap.sig.snps=bedr(input=list(a=res.bed.sorted, b=peaks), method='intersect')
    in_peak=as.numeric(res.bed.sorted %in% overlap.sig.snps)
    pvals=res$"p.value"[match(res.bed.sorted, res$id)]
    #pvals=res$"pvalue"[match(res.bed.sorted, res$id)]
    #pvals=res$"p-value"[match(res.bed.sorted, res$id)]
    mw_res=wilcox.test(pvals ~ in_peak, alternative='greater')
    df=data.frame(W=mw_res$statistic, pval=mw_res$p.value, group=g)
    all.df=rbind(all.df, df)   
}

write.table(all.df, file=out, row.names=F, col.names=T, quote=F, sep='\t')

}


[1] "T4"
[1] "gene"
          SNP  gene      beta statistic
1 22:42475703 SMDT1 -0.213625  -29.4415
2 22:42476754 SMDT1 -0.213625  -29.4415
3 22:42484466 SMDT1 -0.213625  -29.4415
4 22:42487900 SMDT1 -0.213625  -29.4415
5 22:42488160 SMDT1 -0.213625  -29.4415
6 22:42490600 SMDT1 -0.213625  -29.4415
                                      p.value
1 0.00000000000000000000000000000000006144902
2 0.00000000000000000000000000000000006144902
3 0.00000000000000000000000000000000006144902
4 0.00000000000000000000000000000000006144902
5 0.00000000000000000000000000000000006144902
6 0.00000000000000000000000000000000006144902
                                     FDR
1 0.000000000000000000000000000002600136
2 0.000000000000000000000000000002600136
3 0.000000000000000000000000000002600136
4 0.000000000000000000000000000002600136
5 0.000000000000000000000000000002600136
6 0.000000000000000000000000000002600136
[1] 1015529       6
[1] "expr"
[1] "no gene"
[1] 1015529       6
                SNP    gen